# Agent2Agent (A2A) Protocol: Build Interoperable Agents

In this notebook, you'll learn the Agent2Agent (A2A) protocol basics and build two simple interoperable agents that can exchange structured messages. We'll:

- Understand the motivation behind A2A and a minimal message schema
- Configure an OpenAI model and a base `Agent` that speaks A2A
- Implement two example agents (Researcher and Writer) and a simple mediator
- Run a multi-turn exchange end-to-end
- Practice with an exercise and a bonus tool-use pattern

Prerequisites: an environment variable `OPENAI_API_KEY` set with a valid key.


In [1]:
import os

# Try to load from .env file if available (optional)
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not installed, that's okay

assert os.getenv("OPENAI_API_KEY"), "⚠️ Please set OPENAI_API_KEY in your environment!"
print("✅ OK: OPENAI_API_KEY detected")

✅ OK: OPENAI_API_KEY detected


In [6]:
from typing import List, Literal, Optional, Dict, Any
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# quick sanity check (model listing might be blocked; instead do a no-op create)
print("OpenAI client initialized.")

OpenAI client initialized.


## Minimal A2A: Message Schema

We'll use a compact message format inspired by A2A ideas:

- `role`: "system" | "user" | "assistant" | "tool" | "agent"
- `name`: optional identifier of the speaker agent
- `content`: natural language content
- `actions`: optional list of proposed actions (e.g., tool calls)
- `metadata`: optional dict (e.g., routing hints)

Agents exchange lists of these messages. A mediator (router) decides who speaks next.


In [7]:
from pydantic import BaseModel, Field

class A2AMessage(BaseModel):
    role: Literal["system", "user", "assistant", "tool", "agent"]
    content: str
    name: Optional[str] = None
    actions: Optional[List[Dict[str, Any]]] = None
    metadata: Optional[Dict[str, Any]] = None

class AgentConfig(BaseModel):
    name: str
    system_prompt: str = ""
    model: str = "gpt-4o-mini"  # small, fast model suitable for classroom
    temperature: float = 0.2

class AgentResponse(BaseModel):
    message: A2AMessage
    stop: bool = False  # allow agent to signal completion

class Agent:
    def __init__(self, config: AgentConfig, client: OpenAI):
        self.config = config
        self.client = client

    def build_prompt(self, history: List[A2AMessage]) -> List[Dict[str, str]]:
        messages: List[Dict[str, str]] = []
        if self.config.system_prompt:
            messages.append({"role": "system", "content": self.config.system_prompt})
        for m in history:
            # map A2A roles to chat roles where reasonable
            role = m.role if m.role in {"system", "user", "assistant"} else "user"
            name = f"{m.name}: " if m.name else ""
            content = name + m.content
            messages.append({"role": role, "content": content})
        return messages

    def step(self, history: List[A2AMessage]) -> AgentResponse:
        messages = self.build_prompt(history)
        completion = self.client.chat.completions.create(
            model=self.config.model,
            temperature=self.config.temperature,
            messages=[{"role": m["role"], "content": m["content"]} for m in messages],
        )
        content = completion.choices[0].message.content or ""
        response = AgentResponse(
            message=A2AMessage(role="agent", name=self.config.name, content=content)
        )
        return response

In [8]:
# Define two specialized agents that speak through the same A2A interface

researcher = Agent(
    AgentConfig(
        name="Researcher",
        system_prompt=(
            "You are a helpful research assistant. "
            "Summarize facts concisely, cite sources if possible. "
            "When uncertain, ask clarifying questions."
        ),
    ),
    client,
)

writer = Agent(
    AgentConfig(
        name="Writer",
        system_prompt=(
            "You are a clear technical writer. "
            "Transform research notes into a polished, short paragraph."
        ),
    ),
    client,
)

In [9]:
from typing import Callable
from dataclasses import dataclass

@dataclass
class Route:
    # Simple routing rule: after 'Researcher' speaks, 'Writer' responds; then stop
    next_map: Dict[str, Optional[str]]

    def next_speaker(self, current_name: str) -> Optional[str]:
        return self.next_map.get(current_name)

route = Route(next_map={
    "Researcher": "Writer",
    "Writer": None,  # stop after writer responds
})

history: List[A2AMessage] = [
    A2AMessage(role="user", name="Instructor", content="Explain what Agent2Agent (A2A) is, briefly."),
]

# First, the Researcher responds
resp_r = researcher.step(history)
history.append(resp_r.message)
print(f"{resp_r.message.name}:\n{resp_r.message.content}\n\n")

# Then route to Writer, who polishes the response
next_name = route.next_speaker("Researcher")
if next_name == "Writer":
    resp_w = writer.step(history)
    history.append(resp_w.message)
    print(f"{resp_w.message.name}:\n{resp_w.message.content}")

Researcher:
Agent2Agent (A2A) refers to a communication framework or protocol that enables different agents (which could be software agents, AI systems, or robotic entities) to interact and collaborate with each other. A2A facilitates the exchange of information, coordination of tasks, and sharing of resources among agents, allowing them to work together more effectively to achieve common goals. This concept is often applied in fields such as multi-agent systems, distributed artificial intelligence, and robotics.

If you need more specific details or applications of A2A, please let me know!


Writer:
Agent2Agent (A2A) is a communication framework that enables various agents—such as software programs, AI systems, or robotic entities—to interact and collaborate effectively. By facilitating the exchange of information, task coordination, and resource sharing, A2A allows these agents to work together towards common objectives. This concept is particularly relevant in areas like multi-agent

## Exercise: Add a Fact-Checker Agent

- Create a third agent `FactChecker` whose role is to critique the Researcher output and ask for clarifications if needed.
- Update the routing so that the turn order is: Researcher -> FactChecker -> Writer.
- Keep the same `A2AMessage` history so all agents see the conversation context.

Hint: copy the `Agent` instantiation pattern used for `researcher` and `writer` with a system prompt like: "You verify factual claims, request sources, and highlight ambiguities."


## Bonus: Simple Tool-Use via A2A Actions

We can simulate tool-use by allowing an agent to propose an action in `actions`, and a separate tool-runner to execute it. Below is a toy calculator tool the Researcher can call.

Run once to define tool and an agent that may propose actions.


In [10]:
from math import sqrt

TOOLS: Dict[str, Callable[..., Any]] = {}

def tool(name: str):
    def decorator(fn: Callable[..., Any]):
        TOOLS[name] = fn
        return fn
    return decorator

@tool("calculator")
def calculator(expr: str) -> str:
    try:
        # Danger: eval — keep to math-only by removing builtins
        result = eval(expr, {"__builtins__": {}}, {"sqrt": sqrt})
        return str(result)
    except Exception as e:
        return f"ERROR: {e}"

class ToolAwareAgent(Agent):
    def step(self, history: List[A2AMessage]) -> AgentResponse:
        # Let the LLM propose an action via a protocol hint
        prompt_hint = (
            "If you need to compute something, propose an action in JSON as: "
            "ACTION: {\"tool\": \"calculator\", \"input\": \"...\"}. "
            "Otherwise, answer normally."
        )
        extended = history + [A2AMessage(role="system", content=prompt_hint)]
        resp = super().step(extended)

        # crude parse for ACTION: {...}
        import re, json
        match = re.search(r"ACTION:\s*(\{.*\})", resp.message.content, re.DOTALL)
        if match:
            try:
                action = json.loads(match.group(1))
                tool_name = action.get("tool")
                tool_input = action.get("input", "")
                if tool_name in TOOLS:
                    tool_result = TOOLS[tool_name](tool_input)
                    tool_msg = A2AMessage(role="tool", name=tool_name, content=str(tool_result))
                    # Have the agent incorporate the tool result
                    follow_up = super().step(history + [resp.message, tool_msg])
                    return follow_up
            except Exception:
                pass
        return resp

calc_researcher = ToolAwareAgent(
    AgentConfig(
        name="Researcher",
        system_prompt=(
            "You are a research assistant who can optionally use a calculator tool. "
            "When asked to compute, propose an ACTION with a simple expression e.g. '2+2' or 'sqrt(9)'."
        ),
    ),
    client,
)

In [11]:
# Ask the tool-aware researcher a question that requires computation
history_calc: List[A2AMessage] = [
    A2AMessage(role="user", name="Instructor", content="What is sqrt(144) + 10? Answer with reasoning and final number."),
]
resp_calc = calc_researcher.step(history_calc)
print(f"{resp_calc.message.name}:\n{resp_calc.message.content}")

Researcher:
To solve the expression \( \sqrt{144} + 10 \), we first need to calculate \( \sqrt{144} \).

The square root of 144 is 12, since \( 12 \times 12 = 144 \).

Now, we add 10 to this result:

\( 12 + 10 = 22 \).

Thus, the final answer is 22.


In [12]:
# Instantiate FactChecker and run Researcher -> FactChecker -> Writer
fact_checker = Agent(
    AgentConfig(
        name="FactChecker",
        system_prompt=(
            "You verify factual claims, request sources, and highlight ambiguities. "
            "Be concise and constructive; suggest corrections or needed citations."
        ),
    ),
    client,
)

# New route order
route_fc = {
    "Researcher": "FactChecker",
    "FactChecker": "Writer",
    "Writer": None,
}

def run_with_fact_checker(user_prompt: str) -> None:
    convo: List[A2AMessage] = [
        A2AMessage(role="user", name="Instructor", content=user_prompt)
    ]

    # Researcher turn
    resp_r = researcher.step(convo)
    convo.append(resp_r.message)
    print(f"{resp_r.message.name}:\n{resp_r.message.content}\n\n")

    # FactChecker turn
    if route_fc["Researcher"] == "FactChecker":
        resp_f = fact_checker.step(convo)
        convo.append(resp_f.message)
        print(f"{resp_f.message.name}:\n{resp_f.message.content}\n\n")

    # Writer turn
    if route_fc["FactChecker"] == "Writer":
        resp_w = writer.step(convo)
        convo.append(resp_w.message)
        print(f"{resp_w.message.name}:\n{resp_w.message.content}")

# Example run
run_with_fact_checker("Explain what Agent2Agent (A2A) is, briefly.")

Researcher:
Agent2Agent (A2A) refers to a communication framework or protocol that enables different autonomous agents to interact and collaborate with each other. This can involve sharing information, negotiating, or coordinating actions to achieve common goals. A2A systems are often used in fields like artificial intelligence, robotics, and distributed computing, where multiple agents need to work together effectively. The specifics of A2A can vary depending on the context and application, such as in multi-agent systems or decentralized networks. 

If you need more detailed information or specific applications of A2A, please let me know!


FactChecker:
Your explanation of Agent2Agent (A2A) is generally accurate, highlighting its role in enabling communication and collaboration among autonomous agents. However, it would be beneficial to clarify that A2A can encompass various protocols and standards, such as FIPA (Foundation for Intelligent Physical Agents) or other frameworks tailored